In [1]:
from bs4 import BeautifulSoup
import requests, re, json
%run T20_Match_Results_WebScraping.ipynb
import pandas as pd

In [2]:
matches_final =[]
for item in matches:
    matches_final.append("https://www.espncricinfo.com"+item[0])

In [3]:
len(matches_final)

2630

In [4]:
#get player URLS
def get_batting_stats(team,url):
    batting_stats_team =pd.DataFrame(columns=['BATTER','Dismissal', 'R', 'B', 'M', '4s', '6s', 'SR','URL','Match'])
    tbody = team.find_all('tbody')
    rows = tbody[0].find_all('tr')
    data =[]
    team1,team2 = extract_teams(url)
    for index, row in enumerate(rows):
            row_list = row.find_all('td')
            temp = [item.get_text() for item in row_list]
            divs = row_list[0].find_all('div')
            for div in divs:
                anchors = div.find_all('a')
                for anchor in anchors:
                    url = anchor['href']
            temp.append(url)
            temp.append(team1+' vs '+team2)
            data.append(temp)
    data = [item for item in data if len(item) > 7]   
    data = data[:len(data)-2]
    for item in data:
        batting_stats_team.loc[len(batting_stats_team)]=item
    return batting_stats_team

In [5]:
#bowling stats
def get_bowling_stats(team,url):
    bowling_stats_team =pd.DataFrame(columns=['BOWLER','O', 'M', 'R', 'W', 'Econ', 'WD', 'NB','URL','Match'])
    tbody = team.find_all('tbody')
    rows = tbody[0].find_all('tr')
    data =[]
    team1,team2 = extract_teams(url)
    for row in rows:
            row_list = row.find_all('td')
            temp = [item.get_text() for index,item in enumerate(row_list) if index not in [6,7,8]]
            anchors = row_list[0].find_all('a')
            for anchor in anchors:
                url = anchor['href']
            temp.append(url)
            temp.append(team1+' vs '+team2)
            data.append(temp)
            #data.append([item.get_text() for index,item in enumerate(row_list) if index not in [6,7,8]])
    data = [item for item in data if len(item) > 3]   
    for item in data:
        bowling_stats_team.loc[len(bowling_stats_team)]=item
    return bowling_stats_team

In [6]:
import re
\d{5,6}
def extract_teams(url):
    # Define the start and end patterns
    start_pattern = r'\d{5,6}/'
    end_pattern1 = r'-vs-'
    end_pattern2 = r'-'

    # Initialize team names
    team1 = None
    team2 = None
    complex_teams = {'sri': 'sri-lanka', 'new': 'new-zealand', 'west': 'west-indies', 'south': 'south-africa'}

    # Use re.search to find the first match for team1
    try:
        match1 = re.search(start_pattern + '(.*?)' + end_pattern1, url)
        if match1:
            team1_candidate = match1.group(1).lower()
            team1 = complex_teams.get(team1_candidate, match1.group(1))
        else:
            team1 = 'Team 1 not matched, url: ' + url
    except Exception as e:
        team1 = 'Error extracting Team 1: ' + str(e)

    # Use re.search to find the first match for team2
    try:
        match2 = re.search(end_pattern1 + '(.*?)' + end_pattern2, url)
        if match2:
            team2_candidate = match2.group(1).lower()
            team2 = complex_teams.get(team2_candidate, match2.group(1))
        else:
            team2 = 'Team 2 not matched, url: ' + url
    except Exception as e:
        team2 = 'Error extracting Team 2: ' + str(e)

    return team1, team2

In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_and_append_to_csv(match_url):
    try:
        soup = BeautifulSoup(requests.get(match_url).text, "html.parser")
        tables = soup.find_all("table")[0:4]

        # Assuming get_batting_stats and get_bowling_stats return dataframes
        batting_stats_1 = get_batting_stats(tables[0], match_url)
        batting_stats_2 = get_batting_stats(tables[2], match_url)
        bowling_stats_1 = get_bowling_stats(tables[1], match_url)
        bowling_stats_2 = get_bowling_stats(tables[3], match_url)

        # Append to batting_summary.csv
        batting_stats_1.to_csv('batting_summary.csv', mode='a', header=False, index=False)
        batting_stats_2.to_csv('batting_summary.csv', mode='a', header=False, index=False)

        # Append to bowling_summary.csv
        bowling_stats_1.to_csv('bowling_summary.csv', mode='a', header=False, index=False)
        bowling_stats_2.to_csv('bowling_summary.csv', mode='a', header=False, index=False)

        successful_urls.append(match_url)
    except Exception as e:
        print(f"Error at: {match_url} - {e}")
        unsuccessful_urls.append(match_url)

# Define the initial structure of the CSV files with headers
def initialize_csv_files():
    # Create empty DataFrame with the proper columns
    batting_columns = ['BATTER', 'Dismissal', 'R', 'B', 'M', '4s', '6s', 'SR', 'URL', 'Match']
    bowling_columns = ['BOWLER', 'O', 'M', 'R', 'W', 'Econ', 'WD', 'NB', 'URL', 'Match']

    # Write the header to the CSV files
    pd.DataFrame(columns=batting_columns).to_csv('batting_summary.csv', index=False)
    pd.DataFrame(columns=bowling_columns).to_csv('bowling_summary.csv', index=False)

# Initialize the CSV files with headers
initialize_csv_files()

successful_urls = []
unsuccessful_urls = []

for match in matches_remaining:
    extract_and_append_to_csv(match)

print("Successful URLs:", successful_urls)
print("Unsuccessful URLs:", unsuccessful_urls)


Error at: https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2021-22-1267897/afghanistan-vs-india-33rd-match-group-2-1273744/full-scorecard - cannot set a row with mismatched columns
Error at: https://www.espncricinfo.com/series/uganda-t20-tri-series-2021-1275034/uganda-vs-kenya-1st-match-1275041/full-scorecard - cannot set a row with mismatched columns
Error at: https://www.espncricinfo.com/series/pakistan-tour-of-west-indies-2021-1263146/west-indies-vs-pakistan-4th-t20i-1263167/full-scorecard - cannot set a row with mismatched columns
Error at: https://www.espncricinfo.com/series/pakistan-tour-of-west-indies-2021-1263146/west-indies-vs-pakistan-3rd-t20i-1263166/full-scorecard - cannot set a row with mismatched columns
Error at: https://www.espncricinfo.com/series/pakistan-tour-of-west-indies-2021-1263146/west-indies-vs-pakistan-1st-t20i-1263164/full-scorecard - cannot set a row with mismatched columns
Error at: https://www.espncricinfo.com/series/australia-in-new-zealand-202

In [32]:
successful_urls[len(successful_urls)-1]

'https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-africa-region-qualifier-2021-22-1289037/nigeria-vs-tanzania-12th-match-1289053/full-scorecard'

In [33]:
len(successful_urls)

84

In [48]:
len(unsuccessful_urls)

35

In [20]:
#unsuccessful_urls

In [30]:
len(unsuccessful_urls)

3

In [45]:
matches_remaining = matches_final[1127:]

In [40]:
for index,match in enumerate(matches_final):
    if match == str("https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-africa-region-qualifier-2021-22-1289037/nigeria-vs-tanzania-12th-match-1289053/full-scorecard"):
        print(index)
        break

1127


In [ ]:
"https://www.espncricinfo.com/series/west-indies-tour-of-new-zealand-2008-09-366700/new-zealand-vs-west-indies-2nd-t20i-366708/full-scorecard"